Applying the HMM to the sleep dataset
======

Back to the dataset we'll be working with for this tutorial, a Drosophila movement dataset. Data can come in all formats, but a csv file is one of the most common due to its simplicity and intergration with spreadsheets. The data we will be using for this turorial is real, raw data from the Gilestro lab, where we track and record the movement of fruit flies using machine vision. The tracking is able to discern small movements in the fly that can robustly record the flies several times per second giving a multiude of variables to work with.

We'll be using the pandas package to import and store our data in the notebooks. Pandas is a widely used tool in data science, it is built ontop of numpy which we briefly used previously. At the core of pandas is the DataFrame, a table format you will all be familiar wih from spreadsheets. Pandas gives you many tools to manipulate the data before you feed it into any analysis or machine learning tool. As with numpy everything used here will be explained as we use it, but if you'd like to read more into how to use pandas there is a quick tutoiral on their website -> [here](https://pandas.pydata.org/docs/user_guide/10min.html)

In [10]:
# first we need to import pandas
# like numpy it is often imported in a shorthand 
import pandas as pd
import numpy as np

In [58]:
# Pandas has many builtin in functions that can read different data formats into a DataFrame
df = pd.read_csv('/home/lab/Desktop/ReCoDE-HMMs-for-the-discovery-of-behavioural-states/admin/training_data_30.zip')

In [59]:
# def add_nans(data):
#     t_array = data['t'].to_numpy()
#     t_select = np.random.permutation(t_array)[:3]
#     for i in t_select:
#         for q in np.random.permutation(['x', 'y', 'w', 'h', 'max_velocity', 'mean_velocity', 'moving', 'micro', 'walk'])[:4]:
#             data[q][data['t'] == i] = np.nan
#     return data
# df = df.groupby('id', group_keys = False).apply(add_nans)

The Data Structure
=================

The first column is 'id' which contains a unique ID per fly and will allow us to filter and apply methods to just one fly at a time. The next most important variable is 't' or time, as we are working with time series data we must ensure this is strucutred properly i.e. in sequential order and at regular intervals (the later we will go over). The rest are various variables per each timestamp, for this turorial we'll only be interested in 'moving', 'micro', and 'walk'.

Checking for missing data
======

Most real datasets will not be perfectly populated, with tracking dropping out over the course of an experiment. In a dataframe or an array where there is data missing at a timepoint or index this will be represented by a NaN value, which lets methods and functions know there is no data rather than a zero value. However, often analysing packages will throw an error if you feed it NaN values, so it's good practice to check for them first and either remove them or replace then with an approximation.

In [72]:
# Lets filter our dataframe for nan values
# With pandas you can filter the dataframe by the columns
# To filter or slice the dataframe put some square brackets after the dataframe and inside call the column slice 
# For finding NaN values we have to call a method, for other regualr filtering you just use =, <, > and so on

df[df['x'].isnull()]

,id,t,x,y,w,h,max_velocity,mean_velocity,moving,micro,walk
33037,2016-04-04_17-39-22_033aee|03,445260,NaN,0.065451,0.046868,0.019324,7.445241,NaN,NaN,False,NaN
45180,2016-04-04_17-39-22_033aee|04,236160,NaN,NaN,NaN,0.019395,0.756622,0.650191,False,False,NaN
64277,2016-04-04_17-39-22_033aee|05,231810,NaN,0.061041,0.050922,NaN,0.751413,0.657350,NaN,NaN,False
71447,2016-04-04_17-39-22_033aee|05,447090,NaN,NaN,NaN,0.017913,NaN,0.727550,True,True,False
108397,2016-04-04_17-39-22_033aee|07,402270,NaN,0.052370,0.053485,NaN,3.418840,0.886484,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6842546,2016-09-27_10-56-35_053c6b|11,183990,NaN,NaN,0.051491,0.024905,0.702876,NaN,NaN,False,False
6853428,2016-09-27_10-56-35_053c6b|11,510450,NaN,0.062462,NaN,0.019540,16.783354,3.954978,True,NaN,NaN
6873720,2016-09-27_10-56-35_053c6b|13,519690,NaN,NaN,0.050000,NaN,0.715943,NaN,False,False,False
6878121,2016-09-27_10-56-35_053c6b|15,53610,NaN,NaN,NaN,0.019024,0.744524,NaN,False,False,False


In [ ]:
# To break down whats happening we can just call whats inside the brackets, you can see that it is an array (or series in pandas terms) with False or True per row.
# This array then dictates what rows get returned from the whole dataframe, i.e. only the ones that fullfill the argument and are True

df['x'].isnull()

id
2016-04-04_17-39-22_033aee|01    False
2016-04-04_17-39-22_033aee|01    False
2016-04-04_17-39-22_033aee|01    False
2016-04-04_17-39-22_033aee|01    False
2016-04-04_17-39-22_033aee|01    False
                                 ...  
2016-04-04_17-39-05_009aee|20    False
2016-04-04_17-39-05_009aee|20    False
2016-04-04_17-39-05_009aee|20    False
2016-04-04_17-39-05_009aee|20    False
2016-04-04_17-39-05_009aee|20    False
Name: moving, Length: 327031, dtype: bool

In [ ]:
# However, we are not just looking at one column. 
# Luckily with pandas you can filter by multiple columns, all you need to do is put each filter argument in round brackets and then seperate them by an & ("and") or | ("or") logical operator

df[(df['x'].isnull()) | (df['y'].isnull())]

,id,t,x,y,w,h,max_velocity,mean_velocity,moving,micro,walk
0,2016-04-04_17-39-22_033aee|01,31140,NaN,0.069594,0.038829,0.020012,75.662162,25.713480,True,False,True
5185,2016-04-04_17-39-22_033aee|01,186960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17891,2016-04-04_17-39-22_033aee|01,568140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18642,2016-04-04_17-39-22_033aee|01,590670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19233,2016-04-04_17-39-22_033aee|03,31140,NaN,0.057547,0.044205,0.018120,283.712679,48.750972,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
6893554,2016-09-27_10-56-35_053c6b|17,437280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6899959,2016-09-27_10-56-35_053c6b|19,31140,NaN,0.045414,0.049787,0.020724,7.724649,3.438745,True,False,True
6900487,2016-09-27_10-56-35_053c6b|19,47040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6904449,2016-09-27_10-56-35_053c6b|19,166080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Now we want to remove those rows containing NaN values as they aren't providing any information
# When filtering for NaNs above we're selecting for them, adding the ~ opertor tells the filter to look for the opposite, so when NaN is True it now becomes False
df_filtered = df[~ (df['x'].isnull()) | ~ (df['y'].isnull())]

# the new DataFrame now won't have any rows where 'x' and 'y' have NaN values
df_filtered

,id,t,x,y,w,h,max_velocity,mean_velocity,moving,micro,walk
0,2016-04-04_17-39-22_033aee|01,31140,0.269116,0.069594,0.038829,0.020012,75.662162,25.713480,True,False,True
1,2016-04-04_17-39-22_033aee|01,31170,0.606590,0.068019,0.048224,0.020609,27.471271,9.145901,True,False,True
2,2016-04-04_17-39-22_033aee|01,31200,0.398307,0.070464,0.049073,0.020628,19.718721,5.478951,True,False,True
3,2016-04-04_17-39-22_033aee|01,31230,0.469571,0.066383,0.046558,0.020423,20.224544,7.475374,True,False,True
4,2016-04-04_17-39-22_033aee|01,31260,0.260085,0.073667,0.047548,0.020133,34.824007,6.163203,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
6919122,2016-09-27_10-56-35_053c6b|19,606420,0.537450,0.047642,0.052998,0.023141,7.428117,2.037493,True,False,True
6919123,2016-09-27_10-56-35_053c6b|19,606450,0.211436,0.063828,0.048854,0.024929,21.177698,4.470726,True,False,True
6919124,2016-09-27_10-56-35_053c6b|19,606480,0.131377,0.065893,0.041694,0.025711,10.986990,3.057987,True,False,True
6919125,2016-09-27_10-56-35_053c6b|19,606510,0.512140,0.064421,0.054938,0.021951,29.166126,6.249765,True,False,True


## Task:
As stated before for this tutorial we will be focussing on the variables 'moving', 'micro', 'walk'. Now you know how to filter out NaN values apply this to only these columns.

In [ ]:
# To complete 

df = 

## Extra Task:
1) If you're new to pandas (or just want some practice) have a play around with other types of filtering (such as df[df['mean_velocity'] > 5]). It makes a quick and easy way to filter your data and if you're doing the same thing repeatably you can create a function to do it instantly.

2) rather than filtering out the NaN values you can replace them with something else. We could know that tracking drops out when the flies are still for a long time, so we could resonably replace all of 'moving', 'micro', and 'walk' with False.
This can be done with the .fillna method, see here for how to do it -> [fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

Binning the data to a larger time step
======

#Outcome - Why you would want to do it, the benfits and downsides

Its important with Hidden Markov models that any timeseries dataset is complete with no skips due to missing data, as the model will assume the array you're feeding it all has the same time step. One way to do this is to increase the timestep, currently the dataset has a row for every 30 seconds. However, we know from filtering out the NaN values that we will won't have the all. So to recitfy this we will increase the time step to 60, as long as there is at least 1 row out of 2 for the 60 we'll have a perfectly populated dataset. 

Addtionally, doing so will decrease the size of the data, meaning the model will train more quickly. It's always worth trying the model with a few different timesteps to see how this affects the output, then you can pick the one you think is the most representative and quickest to train

In [76]:
# we'll go through it step by step, before wrapping it all in a function
# First we'll need the function floor which rounds down numbers
from math import floor

In [83]:
# first we'll create a new column with the new time step
# lambda functions are an easy way to apply a function per row with a specific column
# We then divide the time by our new time and round down. The end result is multiplied by the new time step giving the minimum time as divisable by the time step given
df['bin_t'] = df['t'].map(lambda t: 60 * floor(t / 60)) # the t represenst each row value for the column 't'
df

,id,t,x,y,w,h,max_velocity,mean_velocity,moving,micro,walk,bin_t
0,2016-04-04_17-39-22_033aee|01,31140,0.269116,0.069594,0.038829,0.020012,75.662162,25.713480,True,False,True,31140
1,2016-04-04_17-39-22_033aee|01,31170,0.606590,0.068019,0.048224,0.020609,27.471271,9.145901,True,False,True,31140
2,2016-04-04_17-39-22_033aee|01,31200,0.398307,0.070464,0.049073,0.020628,19.718721,5.478951,True,False,True,31200
3,2016-04-04_17-39-22_033aee|01,31230,0.469571,0.066383,0.046558,0.020423,20.224544,7.475374,True,False,True,31200
4,2016-04-04_17-39-22_033aee|01,31260,0.260085,0.073667,0.047548,0.020133,34.824007,6.163203,True,False,True,31260
...,...,...,...,...,...,...,...,...,...,...,...,...
6919122,2016-09-27_10-56-35_053c6b|19,606420,0.537450,0.047642,0.052998,0.023141,7.428117,2.037493,True,False,True,606420
6919123,2016-09-27_10-56-35_053c6b|19,606450,0.211436,0.063828,0.048854,0.024929,21.177698,4.470726,True,False,True,606420
6919124,2016-09-27_10-56-35_053c6b|19,606480,0.131377,0.065893,0.041694,0.025711,10.986990,3.057987,True,False,True,606480
6919125,2016-09-27_10-56-35_053c6b|19,606510,0.512140,0.064421,0.054938,0.021951,29.166126,6.249765,True,False,True,606480


You should see in the column 'bin_t' that rows next to each other now share a time step. Now we have that we'll want to pivot or group by this column so all that have the same time stamp are collected together.

In [102]:
# The pandas groupby method does this, all you need to do is call the method with the column you want to pivot by in the brackets
# Then you can tell it what aggregating function you want to call on the columns of interest
df_grouped = df.groupby('bin_t').agg(**{
            'x' : ('x', 'mean'), # before the brackets is the name of the new column, we'll keep it the same
            'y' : ('y', 'mean')  # within the brackets is the column you want to use and the function to apply to it. You have 'mean', 'median', 'max'... ect built in, but you can also use your own functions
})

df_grouped

,x,y
bin_t,,
6960,0.548046,0.055010
7020,0.483673,0.055646
7080,0.502220,0.056058
7140,0.499007,0.056736
7200,0.535441,0.055255
...,...,...
608100,0.486551,0.052753
608160,0.466558,0.052861
608220,0.485503,0.052645


Some of you may have noticed that doing it this way will aggregate our whole dataset and lose the information per fly. To keep this information we can call a groupby with two levels, the first will be the higher level that the data is grouped by first, and the second the one that the functions will be applied to.

In [119]:
# We do exactly the same, but instead of just 'bin_t' we have a list with 'id' first
# Calling it this way on a lot of rows can take a few minutes or more depending on your computer, so don't worry if it takes a while
df_grouped = df.groupby(['id', 'bin_t']).agg(**{
            'x' : ('x', 'mean'),
            'y' : ('y', 'mean')
})
# We need to reset the index as it will have both 'id' and 'bin_t' as the index
df_grouped.reset_index(inplace = True)
# We'll also rename the column 'bin_t' back to 't' for clarity
df_grouped.rename(columns = {'bin_t' : 't'})
df_grouped

## Task:
The Same as before, re-create the steps above but for the columns 'moving', 'micro', 'walk'. Instead of mean use max as we care about the most dominant behaviour in that time window, also it will keep our results as eith True of False which are discrete catorgories.

In [125]:
# To complete 

df = 

Interpolating the data
========

Another method to fill in the gaps in data is interpolation. This is where you determin a value at any given timepoint given the rest of the dataset. If you have just a few points missing the interpolation results can be quite accurate. Here we'll run through the steps to interpolating your data

In [120]:
# First we can check if we have all the data points for each fly
# We'll use this method to check

def check_num_points(data, timestep=60):

    array_time = max(data['t']) - min(data['t'])

    if (array_time / timestep) >= len(data):
        return False
    else:
        return True

df.groupby('id', group_keys=False).apply(check_num_points).value_counts()

True     216
False    165
Name: count, dtype: int64

We can see that nearly 50% of our flies are missing some points, so it's best we move ahead with interpolation.

## Extra Task:
Rather than just returning True or False you can create a function that returns the percentage of points you have from the amount needed. You can then combine this to filter our the flies that have less than 75% of points. 

In [ ]:
    

def interpolate(data, var, step, t_col = 't'):

    id = data['id'].iloc[0]
    sample_seq = np.arange(min(data[t_col]), np.nanmax(data[t_col]), step)
    if len(sample_seq) < 3:
        return None
    f  = interp1d(data[t_col].to_numpy(), data[var].to_numpy())
    return  pd.DataFrame(data = {'id' : id, t_col : sample_seq, var : f(sample_seq)})

# **TO DO**

##  Creating a new column that is a categorical summantion of the other 3
### Outcome - How to do the above

# **Extra Tasks**

## 1. Split the data by Male and Female into seperate dataframes 
## 2. Convert a continuous float column to a descrete categorical column